## Tensor Product at `GPU` is much faster then `CPU`

Most deep learning calculation is held by `Matrix Multiplication`<br>
We will make `Dezero` to use `GPU`

---

- install `cupy` for `GPU` usage
    - Library for parallel calculation using GPU

```python
$ pip install cupy
```

---

If the pip doesn't work and your at `Win10` then try the following

```python
$ conda install -c conda-forge cupy cudatoolkit
```

### CUDA / cuDNN

- Check the appropriate CUDA version for your graphic card
    - https://en.wikipedia.org/wiki/CUDA#Version_features_and_specifications
- Install CUDA
    - https://developer.nvidia.com/cuda-toolkit-archive
- Install cuDNN
    - https://developer.nvidia.com/cudnn
    - Move files to `C:\Program Files\NVIDIA GPU Computing Toolkit\CUDA\v10.1`

## `Numpy` & `Cupy`

Both usage are almost the same :)

In [2]:
import cupy as cp

x = cp.arange(6).reshape(2, 3)
print(x)

y = x.sum(axis=1)
print(y)

[[0 1 2]
 [3 4 5]]
[ 3 12]


In [3]:
type(y)

cupy.core.core.ndarray

We can send data back and forth
- `cp.asarray` - **Main Memory** -> **GPU Memory**
- `cp.asnumpy` - **GPU Memory** -> **Main Memory**

In [4]:
import numpy as np
import cupy as cp

# numpy -> cupy
n = np.array([1, 2, 3])
c = cp.asarray(n)

assert type(c) == cp.ndarray

In [5]:
# cupy -> numpy
c = cp.array([1, 2, 3])
n = cp.asnumpy(c)

assert type(n) == np.ndarray

### Caution

This memory movement could be a bottleneck when we treat big data.<br>
It's important to code the memory movement less as possible

## `cp.get_array_module`

Tell is this tensor `numpy` or `cupy`

In [6]:
# numpy array
x = np.array([1, 2, 3])
xp = cp.get_array_module(x)
assert xp == np

# cupy array
x = cp.array([1, 2, 3])
xp = cp.get_array_module(x)
assert xp == cp

We can write **compatible code** for both `numpy` and `cupy` with **`cp.get_array_module`** :)

```python
xp = cp.get_array_module(x)

y = xp.sin(x)
```

## Cuda Module

Compatible with the computer which doesn't have `GPU` or `cupy`

In [7]:
import numpy as np

gpu_enable  = True

try:
    import cupy as cp
    cupy = cp
except ImportError:
    gpu_enable = False

### We are making a wrapper functions of `cupy` to correspond the situation when `cupy` is not installed

In [8]:
from dezero import Variable

def get_array_module(x):
    if isinstance(x, Variable):
        x = x.data
        
    if not gpu_enable:
        return np
    
    xp  = cp.get_array_module(x)
    return xp

def as_numpy(x):
    if isinstance(x, Variable):
        x = x.data
    
    # This code is necessary when cupy is not installed
    if np.isscalar(x):
        return np.array(x)
    elif isinstance(x, np.ndarray):
        return x
    
    return cp.asnumpy(x)

def as_cupy(x):
    if isinstance(x, Variable):
        x = x.data
        
    if not gpu_enable:
        raise Exception('Cannot load Cupy. Install Cupy.')
        
    return cp.asarray(x)

In [9]:
cp.asnumpy(None)

array(None, dtype=object)

In [10]:
np.array(None)

array(None, dtype=object)

In [11]:
np.isscalar(np.array(1)), np.isscalar(1)

(False, True)

In [12]:
np.array(np.array([1, 2, 3]))

array([1, 2, 3])

In [13]:
np.array(cp.array([1, 2, 3]))

ValueError: object __array__ method not producing an array

In [14]:
cp.array(np.array([1, 2, 3]))

array([1, 2, 3])

In [15]:
cp.array(cp.array([1, 2, 3]))

array([1, 2, 3])

only using `cp.asnumpy`

In [16]:
cp.asnumpy(1)

array(1)

In [17]:
cp.asnumpy(np.array([1, 2, 3]))

array([1, 2, 3])

In [18]:
cp.asnumpy(cp.array([1, 2, 3]))

array([1, 2, 3])

only using `cp.asarray`

In [19]:
cp.asarray(1)

array(1)

In [20]:
cp.asarray(cp.array([1, 2, 3]))

array([1, 2, 3])

In [21]:
cp.asarray(np.array([1, 2, 3]))

array([1, 2, 3])

## Modify `dezero` for `cupy` compatible
- Variable
- Layer
- DataLoader

```python
...

try:
    import cupy
    array_types = (np.ndarray, cupy.ndarray)
except ImportError:
    array_types = (np.ndarray)

    
class Variable:

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, array_types):
                raise TypeError(f'{type(data)} is not supported')

    def backward(self, retain_grad=False, create_graph=False):
        if self.grad is None:
            # self.grad = np.ones_like(self.data)
            xp = dezero.cuda.get_array_module(self.data)
            self.grad = Variable(xp.ones_like(self.data))

    ...
    
    -> add new functions!
    
    def to_cpu(self):
        if self.data is not None:
            self.data = dezero.cuda.as_numpy(self.data)
            
    def to_gpu(self):
        if self.data is not None:
            self.data = dezero.cuda.as_cupy(self.data)
```

Now for the `Layer`

```python
class Layer:
    ...
    
    def to_cpu(self):
        for param in self.params():
            param.to_cpu()
        
    def to_gpu(self):
        for param in self.params():
            param.to_gpu()
```

And finally for the `DataLoader`

```python
import numpy as np
from dezero import cuda

class DataLoader:
    def __init__(self, dataset, batch_size, shuffle=True, gpu=False):
        self.dataset = dataset
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.data_size = len(dataset)
        self.max_iter = math.ceil(self.data_size / batch_size)
        self.gpu = gpu

        self.reset()

    def __next__(self):
        
        ...
        
        xp = cuda.cupy if self.gpu else np
        x = xp.array([example[0] for example in batch])
        y = xp.array([example[1] for example in batch])

        self.iteration += 1

        return x, y

    def to_cpu(self):
        self.gpu = False
        
    def to_gpu(self):
        self.gpu = True
```

### Modify `np.xxx` usage

change the functions for the compatibility with `cupy` and `numpy`

```python
import dezero import cuda

xp = cuda.get_array_module(x)
y = xp.sin(x)
```

- **`functions.py`**
    - Sin
    - Cos
    - Tanh
    - Exp
    - Log
    
    - GetItemGrad
        - numpy - `np.add.at`
        - cupy - `cp.scatter_add`
    - Sigmoid
    - ReLU
    - Softmax
    - SoftmaxCrossEntropy
    - logsoftmax
    - Clip
    
- **`layers.py`**
    - Linear
    
- **`optimizers.py`**
    - MomentumSGD
    - AdaGrad
    - RMSProp
    - AdaDelta
    - Adam

**cp.scatter_add**

In [22]:
import cupy as cp

test = cp.array([0, 0, 0])
cp.scatter_add(test, 1, 1)
test

array([0, 1, 0])

### Modify `core.py`

change the functions for the compatibility with `cupy` and `numpy`

```python
def as_array(x, array_module=np):
    if np.isscalar(x):
        return array_module.array(x)
    return x
```

In [23]:
# only scalar value will be treated as True
np.isscalar(cp.array(1))

False

In [24]:
np.isscalar(1)

True

### arithmetic operations

```python
def add(x0, x1):
    x1 = as_array(x1, array_module=dezero.cuda.get_array_module(x0.data))
    return Add()(x0, x1)
```

- add
- sub
- rsub
- mul
- div
- rdiv

## `MNIST` with `GPU`

In [25]:
import time
import math
import numpy as np

import dezero
from dezero import optimizers
from dezero import DataLoader

import dezero.functions as F
from dezero.models import MLP
from dezero.datasets import MNIST

In [26]:
max_epoch = 5
batch_size = 100
hidden_size = 1000

train_set = MNIST(train=True)
test_set = MNIST(train=False)

train_loader = DataLoader(train_set, batch_size)
test_loader = DataLoader(test_set, batch_size, shuffle=False)

model = MLP((hidden_size, 10))
optimizer = optimizers.SGD(lr=0.1).setup(model)

### with `gpu`

In [27]:
if dezero.cuda.gpu_enable:
    train_loader.to_gpu()
    test_loader.to_gpu()
    model.to_gpu()

In [28]:
epoch_list = []

train_loss_list = []
test_loss_list = []

train_acc_list = []
test_acc_list = []


for epoch in range(max_epoch):
    start = time.time()
    sum_loss, sum_acc = 0, 0
    
    for x, y in train_loader:
        
        y_pred = model(x)
        
        loss = F.softmax_cross_entropy(y_pred, y)
        acc = F.accuracy(y_pred, y)
        
        model.cleargrads()
        loss.backward()
        optimizer.update()
        
        sum_loss += float(loss.data) * len(y)
        sum_acc += float(acc.data) * len(y)
    
    avg_loss = sum_loss / len(train_set)
    avg_acc = sum_acc / len(train_set)

    train_loss_list.append(avg_loss)
    train_acc_list.append(avg_acc)

    elasped_time = time.time() - start
    
    print('epoch : {}'.format(epoch + 1))
    print('train loss: {:.4f}, accuracy: {:.4f}, time: {:.4f}[sec]'.format(avg_loss, avg_acc, elasped_time))
    
    sum_loss, sum_acc = 0, 0
    
    with dezero.no_grad():
        for x, y in test_loader:
            y_pred = model(x)
            
            
            loss = F.softmax_cross_entropy(y_pred, y)
            acc = F.accuracy(y_pred, y)
            
            sum_loss += float(loss.data) * len(y)
            sum_acc += float(acc.data) * len(y)
            
    avg_loss = sum_loss / len(test_set)
    avg_acc = sum_acc / len(test_set)            
    
    test_loss_list.append(avg_loss)
    test_acc_list.append(avg_acc)

    print('test loss: {:.4f}, accuracy: {:.4f}'.format(avg_loss, avg_acc))
    
    epoch_list.append(epoch + 1)    

epoch : 1
train loss: 0.9684, accuracy: 0.7243, time: 5.5470[sec]
test loss: 0.4542, accuracy: 0.8594
epoch : 2
train loss: 0.4021, accuracy: 0.8846, time: 4.4742[sec]
test loss: 0.3474, accuracy: 0.9031
epoch : 3
train loss: 0.3508, accuracy: 0.8975, time: 4.3783[sec]
test loss: 0.3279, accuracy: 0.9068
epoch : 4
train loss: 0.3297, accuracy: 0.9045, time: 4.3793[sec]
test loss: 0.3092, accuracy: 0.9109
epoch : 5
train loss: 0.3157, accuracy: 0.9078, time: 4.4072[sec]
test loss: 0.2955, accuracy: 0.9142


### with `cpu`

In [29]:
train_loader.to_cpu()
test_loader.to_cpu()
model.to_cpu()

In [30]:
epoch_list = []

train_loss_list = []
test_loss_list = []

train_acc_list = []
test_acc_list = []


for epoch in range(max_epoch):
    start = time.time()
    sum_loss, sum_acc = 0, 0
    
    for x, y in train_loader:
        
        y_pred = model(x)
        
        loss = F.softmax_cross_entropy(y_pred, y)
        acc = F.accuracy(y_pred, y)
        
        model.cleargrads()
        loss.backward()
        optimizer.update()
        
        sum_loss += float(loss.data) * len(y)
        sum_acc += float(acc.data) * len(y)
    
    avg_loss = sum_loss / len(train_set)
    avg_acc = sum_acc / len(train_set)

    train_loss_list.append(avg_loss)
    train_acc_list.append(avg_acc)

    elasped_time = time.time() - start
    
    print('epoch : {}'.format(epoch + 1))
    print('train loss: {:.4f}, accuracy: {:.4f}, time: {:.4f}[sec]'.format(avg_loss, avg_acc, elasped_time))
    
    sum_loss, sum_acc = 0, 0
    
    with dezero.no_grad():
        for x, y in test_loader:
            y_pred = model(x)
            
            
            loss = F.softmax_cross_entropy(y_pred, y)
            acc = F.accuracy(y_pred, y)
            
            sum_loss += float(loss.data) * len(y)
            sum_acc += float(acc.data) * len(y)
            
    avg_loss = sum_loss / len(test_set)
    avg_acc = sum_acc / len(test_set)            
    
    test_loss_list.append(avg_loss)
    test_acc_list.append(avg_acc)

    print('test loss: {:.4f}, accuracy: {:.4f}'.format(avg_loss, avg_acc))
    
    epoch_list.append(epoch + 1)    

epoch : 1
train loss: 0.3065, accuracy: 0.9115, time: 5.4395[sec]
test loss: 0.2929, accuracy: 0.9166
epoch : 2
train loss: 0.2989, accuracy: 0.9141, time: 5.5671[sec]
test loss: 0.2818, accuracy: 0.9201
epoch : 3
train loss: 0.2928, accuracy: 0.9153, time: 5.6032[sec]
test loss: 0.2790, accuracy: 0.9200
epoch : 4
train loss: 0.2875, accuracy: 0.9170, time: 5.8783[sec]
test loss: 0.2749, accuracy: 0.9216
epoch : 5
train loss: 0.2822, accuracy: 0.9193, time: 5.7746[sec]
test loss: 0.2750, accuracy: 0.9225


### `gpu` - 4.5 sec,  `cpu` - 5.5 sec

There is a small difference(`1 sec`) but not that big here :0